In [34]:

from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import geocoder # import geocoder
import types
import folium
from botocore.client import Config
import ibm_boto3
import requests
from pandas.io.json import json_normalize

<h3> Creating DataFrame

In [3]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
html = urlopen(url)
body= BeautifulSoup(html,'lxml')

table= body.find_all("table")[0]
df = pd.read_html(str(table))[0]
print("Before Removing Not Assigned:",df.shape)

df=df[df["Borough"]!="Not assigned"].reset_index(drop=True)
print("After Removing Not Assigned:",df.shape)
idx=df["Neighbourhood"]=="Not assigned"

df.loc[idx,"Neighbourhood"]=df["Borough"]

# print("Value which got replaced\n:",df[df["Neighbourhood"]=="Not Assigned"])

df_grp=df.groupby(by=["Postcode","Borough"])["Neighbourhood"].apply(','.join).reset_index()
print("Final Shape ",df_grp.shape)
df_grp.head()

Before Removing Not Assigned: (287, 3)
After Removing Not Assigned: (210, 3)
Final Shape  (103, 3)


,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


<h3> Adding Longitude and Latitude in the DataFrame

In [5]:
def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_d13087807b0347c2a99293bfd1b8c452 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='e2fSUi8dL3FGZYHlHjQ-s0A5p9ZkG9k5yq5eDQbk6OyK',
    ibm_auth_endpoint="https://iam.ng.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_d13087807b0347c2a99293bfd1b8c452.get_object(Bucket='dataclustering-donotdelete-pr-vtlj4gybkeelhm',Key='Geospatial_Coordinates.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

# If you are reading an Excel file into a pandas DataFrame, replace `read_csv` by `read_excel` in the next statement.
df_data_0 = pd.read_csv(body)
df_data_0.head()


df_new=pd.merge(df_grp,df_data_0,how="inner",left_on="Postcode", right_on="Postal Code")
df_new.drop(columns="Postal Code",inplace=True)
print("Shape of the dataframe:",df_new.shape)
df_new.head()


Shape of the dataframe: (103, 5)


,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


<h3> DataFrame with rows only for Toronto values

In [20]:
df_toronto=df_new[df_new["Borough"].str.contains("Toronto")].reset_index(drop=True)
df_toronto.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West,India Bazaar",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [21]:
#Toronto latitude and longitude
latitude=43.6
longitude=-79.3
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Borough'], df_toronto['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

<h3> Define FourSquare Credentials in our Dataframe

In [22]:
CLIENT_ID = 'XBBTYWTYMYK0XHBDXYTUQ34PQ1HUBBE0LPMF3Z05W4PX0XCD' # your Foursquare ID
CLIENT_SECRET = '3NGRPBZAY2BFXJP0Y3OK22UIT4LPD0UTIM4DS5FOWPIWGO1F' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: XBBTYWTYMYK0XHBDXYTUQ34PQ1HUBBE0LPMF3Z05W4PX0XCD
CLIENT_SECRET:3NGRPBZAY2BFXJP0Y3OK22UIT4LPD0UTIM4DS5FOWPIWGO1F


In [23]:
df_toronto.loc[0,"Neighbourhood"]

'The Beaches'

In [25]:
neighborhood_latitude = df_toronto.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df_toronto.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = df_toronto.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of The Beaches are 43.67635739999999, -79.2930312.


<h3>1. Now, let's get the top 100 venues that are in the beaches Marble Hill within a radius of 500 meters.</h3>

In [27]:
LIMIT=100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=XBBTYWTYMYK0XHBDXYTUQ34PQ1HUBBE0LPMF3Z05W4PX0XCD&client_secret=3NGRPBZAY2BFXJP0Y3OK22UIT4LPD0UTIM4DS5FOWPIWGO1F&v=20180605&ll=43.67635739999999,-79.2930312&radius=500&limit=100'

In [31]:
results = requests.get(url).json()

In [32]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [35]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Glen Manor Ravine,Trail,43.676821,-79.293942
1,The Big Carrot Natural Food Market,Health Food Store,43.678879,-79.297734
2,Grover Pub and Grub,Pub,43.679181,-79.297215
3,Upper Beaches,Neighborhood,43.680563,-79.292869


In [36]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

4 venues were returned by Foursquare.


<h3>2. Explore Neighourhoods in the Beaches area

In [37]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [39]:
# type your answer here

toronto_venues = getNearbyVenues(names=df_toronto['Neighbourhood'],
                                   latitudes=df_toronto['Latitude'],
                                   longitudes=df_toronto['Longitude']
                                  )



The Beaches
The Danforth West,Riverdale
The Beaches West,India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park,Summerhill East
Deer Park,Forest Hill SE,Rathnelly,South Hill,Summerhill West
Rosedale
Cabbagetown,St. James Town
Church and Wellesley
Harbourfront
Ryerson,Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide,King,Richmond
Harbourfront East,Toronto Islands,Union Station
Design Exchange,Toronto Dominion Centre
Commerce Court,Victoria Hotel
Roselawn
Forest Hill North,Forest Hill West
The Annex,North Midtown,Yorkville
Harbord,University of Toronto
Chinatown,Grange Park,Kensington Market
CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place,Underground city
Christie
Dovercourt Village,Dufferin
Little Portugal,Trinity
Brockton,Exhibition Place,Parkdale Village
High Park,The Junction South
Parkdale,Roncesvalles
Runnymede

In [41]:
print(toronto_venues.shape)
print(toronto_venues.head())

(1715, 7)
                  Neighborhood  Neighborhood Latitude  Neighborhood Longitude  \
0                  The Beaches              43.676357              -79.293031   
1                  The Beaches              43.676357              -79.293031   
2                  The Beaches              43.676357              -79.293031   
3                  The Beaches              43.676357              -79.293031   
4  The Danforth West,Riverdale              43.679557              -79.352188   

                                Venue  Venue Latitude  Venue Longitude  \
0                   Glen Manor Ravine       43.676821       -79.293942   
1  The Big Carrot Natural Food Market       43.678879       -79.297734   
2                 Grover Pub and Grub       43.679181       -79.297215   
3                       Upper Beaches       43.680563       -79.292869   
4                            Pantheon       43.677621       -79.351434   

      Venue Category  
0              Trail  
1  Health Fo

<h3> How many unique Categories are their from returned venues

In [42]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 233 uniques categories.
